In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = '/content/drive/MyDrive/IMG_20230918_081440_475.jpg'
img = keras.utils.load_img(img_path, target_size=(224, 224))
x = keras.utils.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

print('Predicted:', decode_predictions(preds, top=3)[0])






35363/35363 [==============================] - 0s 0us/step
Predicted: [('n02791270', 'barbershop', 0.49492624), ('n04591157', 'Windsor_tie', 0.17758378), ('n02883205', 'bow_tie', 0.09248895)]


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions #Fixed typo here
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np

# Load full InceptionV3 model (with top layer)
inception_full_model = InceptionV3(weights='imagenet', include_top=True) #Fixed typo here

# Load InceptionV3 without the top layer (for transfer learning)
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer for transfer learning
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer (assuming 10 classes for CIFAR-10)
predictions = Dense(10, activation='softmax')(x)

# This is the model to train
model = Model(inputs=base_model.input, outputs=predictions)

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Resize CIFAR-10 images to 299x299 (InceptionV3 input size)
def resize_images(images):
    resized_images = tf.image.resize(images, (299, 299))
    return resized_images

x_train = resize_images(x_train)
x_test = resize_images(x_test)

# Normalize the data to values between -1 and 1 (InceptionV3 preprocesses images this way)
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Convert arrays to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(10000)
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# First predict without any transfer learning (using full InceptionV3)
def predict_without_transfer(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Predict using the full InceptionV3 model (pre-trained)
    preds = inception_full_model.predict(x)
    print('Predictions without transfer learning:')
    print(decode_predictions(preds, top=3)[0])  # Top 3 predictions

# Apply transfer learning
def apply_transfer_learning(epochs=2, batch_size=32):
    # Freeze all layers in the base model (InceptionV3 without top)
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model (after freezing)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the top layers of the model (the newly added layers)
    model.fit(
        train_dataset,
        epochs=epochs,  # Set fewer epochs here
        validation_data=val_dataset
    )

    # Unfreeze some top layers and fine-tune
    for layer in base_model.layers[-20:]:
        layer.trainable = True

    # Recompile the model with a low learning rate
    model.compile(optimizer=Adam(learning_rate=0.000), loss='categorical_crossentropy', metrics=['accuracy'])

    # Continue training with a lower learning rate
    model.fit(
        train_dataset,
        epochs=epochs,  # Set fewer epochs here
        validation_data=val_dataset
    )



170498071/170498071 [==============================] - 2s 0us/step


In [11]:

from tensorflow.keras.preprocessing import image
import numpy as np

def predict_with_transfer(image_path):
  img = image.load_img(image_path, target_size=(299, 299))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  preds = model.predict(x)
  predicted_class = np.argmax(preds[0])

  cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'handsome man']
  predicted_label = cifar10_labels[predicted_class]

  print('Predictions with transfer learning:')
  print(f'Predicted class: {predicted_label}')

# Path to the image for prediction (with transfer learning)
image_path = '/content/drive/MyDrive/IMG_20230918_081440_475.jpg'

# Predict with transfer learning and print the CIFAR-10 label
predict_with_transfer(image_path)


1/1 [==============================] - 0s 77ms/step
Predictions with transfer learning:
Predicted class: handsome man
